In [ ]:
%load_ext blackcellmagic

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import time
from itertools import chain

In [ ]:
data_path = "data/mvtec_anomaly_detection"
classes = list(filter(lambda d:os.path.isdir(os.path.join(data_path,d)),  os.listdir(data_path)))
classes

In [ ]:
object_categories = ["bottle","cable","capsule","metal_nut","pill","screw","tile","toothbrush","transistor","zipper"]

In [ ]:
def read_images_and_yield(dir_path):
    """Reads images per category for given directory path.
    Returns images as a generator object in intervall[0,1] rgb-format"""
    # data = np.empty(shape=(0,1,1,3),dtype=np.int8)
    cur_shape= None
    for f in os.listdir(dir_path):
        file_path =os.path.join(dir_path,f)
        if os.path.isfile(file_path):
            img = cv2.imread(file_path)   # reads an image in the BGR format
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # BGR -> RGB
            if not cur_shape:
                cur_shape = img.shape 
            yield img / 255

In [ ]:
def np_to_img(x):
    plt.imshow(x)
    plt.show()

In [ ]:
normal_train_data = {}
normal_test_data = {}
anomaly_test_data = {}
for cat in object_categories:
    normal_path = os.path.join(data_path, cat, "train", "good")
    normal_train_data[cat] = read_images_and_yield(normal_path)
    
    test_path = os.path.join(data_path,cat,"test")
    normal_test_data[cat] = read_images_and_yield(os.path.join(test_path,"good"))
    anomaly_test_data[cat] = None
    for p in os.listdir(test_path):
        if p != "good":#    
            gen = read_images_and_yield(os.path.join(test_path,p))
            anomaly_test_data[cat] = chain(anomaly_test_data[cat],gen) if anomaly_test_data[cat] != None else gen
normal_train_data,normal_test_data, anomaly_test_data

In [ ]:
# "materializing" the generator
start = time.perf_counter()
for a in anomaly_test_data["cable"]:
    np_to_img(a)
print(time.perf_counter() -start)
len(list(anomaly_test_data["cable"]))
